In [ ]:
#imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, fbeta_score, confusion_matrix, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTENC
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Embedding, Flatten
import tensorflow as tf

In [ ]:
# Import data
data_source = 'https://raw.githubusercontent.com/Rapo-zevs/eda/main/predictive_maintenance.csv'
data = pd.read_csv(data_source)

# First checks
data.info()
print('Number of null values is: {}'.format(data.isnull().sum().sum()))